# GIFT Direct Eigenvalue Computation (Optimized)

**Méthode directe optimisée** : Pour une métrique séparable g = g₁×g₂×...×g₇

Les valeurs propres 7D sont des **sommes** de valeurs propres 1D:
- λ_total = λ₁⁽¹⁾ + λ₂⁽²⁾ + ... + λ₇⁽⁷⁾
- Premier état excité: λ₁(7D) = min_d λ₁(L_d)

On résout 7 petits problèmes 1D au lieu d'un énorme problème 7D!

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
print('Ready!')

In [ ]:
def build_laplacian_1d(n, L, g_func):
    """
    Build 1D curved Laplacian on periodic domain [0, L]
    
    Δ_g f = (1/√g) d/dx (√g/g × df/dx) = (1/√g) d/dx (1/√g × df/dx)
    
    For metric g(x), this is a negative semi-definite operator.
    """
    h = L / n
    x = np.linspace(0, L, n, endpoint=False)
    g = g_func(x)
    sqrt_g = np.sqrt(g)
    
    # Coefficients at half-points
    A_plus = 2.0 / (sqrt_g + np.roll(sqrt_g, -1))   # A at x + h/2
    A_minus = 2.0 / (sqrt_g + np.roll(sqrt_g, 1))   # A at x - h/2
    
    # Sparse tridiagonal with periodic wrapping
    diag_main = -(A_plus + A_minus) / h**2
    diag_upper = A_plus / h**2
    diag_lower = A_minus / h**2
    
    # Build sparse matrix
    L_mat = diags([diag_main, diag_upper[:-1], diag_lower[1:], 
                   [diag_lower[0]], [diag_upper[-1]]],
                  [0, 1, -1, n-1, -(n-1)], format='csr')
    return L_mat, x, g

def get_lambda1_1d(n, L, g_func):
    """Get first non-zero eigenvalue of 1D Laplacian"""
    L_mat, _, _ = build_laplacian_1d(n, L, g_func)
    # Eigenvalues are negative; we want the smallest |λ| > 0
    evals, _ = eigsh(L_mat, k=3, which='LM', sigma=0)
    evals = np.sort(evals)  # Most negative to least negative
    # λ₀ ≈ 0, λ₁ is first non-zero (most negative after 0)
    return -evals[-2]  # Return positive value

def get_lambda1_7d_separable(n, L, g_funcs):
    """
    For separable metric, λ₁(7D) = min over d of λ₁(1D, direction d)
    
    This is because the first excited state has one direction excited
    and all others in ground state (λ=0).
    """
    lambda1_list = []
    for d, g_func in enumerate(g_funcs):
        lam = get_lambda1_1d(n, L, g_func)
        lambda1_list.append(lam)
    return min(lambda1_list), lambda1_list

In [ ]:
# Validate 1D Laplacian on flat torus
print('='*60)
print('VALIDATION: 1D Flat Laplacian')
print('On [0, 2π] with g=1: λ₁ = 1 (from sin(x), cos(x))')
print('='*60)

def flat_metric(x):
    return np.ones_like(x)

for n in [50, 100, 200, 500]:
    lam = get_lambda1_1d(n, 2*np.pi, flat_metric)
    print(f'n={n:4d}: λ₁ = {lam:.8f}, error = {abs(lam-1)*100:.4f}%')

print('\n=> 1D solver is accurate!')

In [ ]:
# Test 1: Flat torus T^7 (calibration)
print('='*60)
print('TEST 1: Flat Torus T^7 (calibration)')
print('Expected: λ₁ = 1.0 (from e^{ix_j})')
print('='*60)

n = 200  # Fine grid for 1D (fast!)
L = 2 * np.pi
g_funcs = [flat_metric] * 7

lambda_1, lam_list = get_lambda1_7d_separable(n, L, g_funcs)
print(f'\nλ₁(1D) for each direction: {[f"{l:.6f}" for l in lam_list]}')
print(f'λ₁(7D) = min = {lambda_1:.6f}')
print(f'Expected: 1.0')
print(f'Error: {abs(lambda_1-1)*100:.4f}%')

In [ ]:
# Test 2: Scaled torus (sanity check)
# If g = c², then λ₁ = 1/c² (metric scales eigenvalues)
# For c² = H*/14, we get λ₁ = 14/H*
print('='*60)
print('TEST 2: Scaled Torus (sanity check)')
print('g = H*/14 => λ₁ = 14/H*')
print('='*60)

n = 200
L = 2 * np.pi

for H_star in [56, 72, 99, 104]:
    c_sq = H_star / 14.0
    
    def scaled_metric(x, c2=c_sq):
        return c2 * np.ones_like(x)
    
    g_funcs = [lambda x, c2=c_sq: c2 * np.ones_like(x)] * 7
    lambda_1, _ = get_lambda1_7d_separable(n, L, g_funcs)
    
    expected = 14.0 / H_star
    error = abs(lambda_1 - expected) / expected * 100
    print(f'H*={H_star:3d}: λ₁={lambda_1:.6f}, 14/H*={expected:.6f}, error={error:.4f}%')

In [ ]:
# Test 3: TCS-like metric (the REAL test)
# Neck direction: g(t) = cosh²((t-T/2)/T₀) where T ~ √H*, T₀ = T/3
# Other directions: flat
# This is a VARIABLE metric - NOT designed to give 14/H*
print('='*60)
print('TEST 3: TCS Metric (variable, the REAL test)')
print('Neck: g(t) = cosh²((t-T/2)/T₀), T=√H*, T₀=T/3')
print('='*60)

n = 500  # High resolution for accuracy
results_tcs = []

for name, b2, b3 in [('K7', 21, 77), ('J1', 12, 43), ('J4', 0, 103), ('Kov', 0, 71)]:
    H_star = b2 + b3 + 1
    T = np.sqrt(H_star)
    T0 = T / 3
    
    # Neck metric: g(t) = cosh²((t-T/2)/T₀)
    # Domain is [0, T], but we use [0, 2π] and scale
    L_neck = T
    
    def neck_metric(x, T=T, T0=T0):
        """TCS neck profile on domain [0, T]"""
        return np.cosh((x - T/2) / T0)**2
    
    # Compute λ₁ for neck direction (domain [0, T])
    lambda_neck = get_lambda1_1d(n, L_neck, neck_metric)
    
    # Flat directions have λ₁ = 1 on [0, 2π]
    # But if we normalize all to same domain T, flat gives λ₁ = (2π/T)²
    # So neck eigenvalue competes with flat eigenvalues
    
    # For separable TCS: λ₁(7D) = min(λ_neck, λ_flat)
    # where λ_flat = (2π/T)² for each flat direction on domain T
    L_flat = 2 * np.pi
    lambda_flat = get_lambda1_1d(n, L_flat, flat_metric)
    
    lambda_1 = min(lambda_neck, lambda_flat)
    
    expected = 14.0 / H_star
    product = lambda_1 * H_star
    
    print(f'{name}: H*={H_star:3d}, λ_neck={lambda_neck:.6f}, λ_flat={lambda_flat:.6f}')
    print(f'       λ₁(7D)={lambda_1:.6f}, λ₁×H*={product:.2f} (target: 14)')
    print()
    results_tcs.append((name, H_star, lambda_1, lambda_neck, lambda_flat, product))

In [ ]:
# Summary table
print('='*60)
print('SUMMARY: TCS Metric Results')
print('='*60)
print(f"{'Manifold':<8} {'H*':>4} {'λ_neck':>10} {'λ_flat':>10} {'λ₁×H*':>10}")
print('-'*50)
products = []
for name, H_star, lam1, lam_n, lam_f, prod in results_tcs:
    winner = 'neck' if lam_n < lam_f else 'flat'
    print(f'{name:<8} {H_star:>4} {lam_n:>10.4f} {lam_f:>10.4f} {prod:>10.2f} ({winner})')
    products.append(prod)
print('-'*50)
print(f"{'Mean':<8} {'':<4} {'':<10} {'':<10} {np.mean(products):>10.2f}")
print(f"{'Std':<8} {'':<4} {'':<10} {'':<10} {np.std(products):>10.2f}")
print(f'\nTarget: λ₁ × H* = 14')

In [ ]:
# Analyse and conclusion
print('='*60)
print('ANALYSIS')
print('='*60)

mean_prod = np.mean(products)
print(f'\nMean λ₁ × H* = {mean_prod:.2f}')
print(f'GIFT prediction = 14.00')
print(f'Deviation = {abs(mean_prod - 14)/14*100:.1f}%')

# Interpretation
print('\n--- INTERPRETATION ---')
print('''
Le modèle TCS séparable (neck + tore plat) ne capture pas la 
géométrie G₂ complète. Les valeurs propres dépendent de :
1. La longueur du neck T ~ √H*
2. Le profil métrique cosh² dans le neck  
3. La taille du tore transverse

Pour tester λ₁ × H* = 14, il faudrait :
- Une métrique G₂ COMPLÈTE (non séparable)
- Le bon ratio entre neck et sections transverses
- Possiblement la métrique de Ricci-flat exacte
''')

# Key insight
print('--- KEY INSIGHT ---')
print('λ_neck dépend du profil cosh² sur [0, √H*]')
print('λ_flat = 1 (constant)')
print('Le minimum détermine λ₁(7D)')
print()
print('Pour que λ₁×H* = 14, il faut λ₁ = 14/H*')
print('Avec H*=99: λ₁ ≈ 0.1414')
print('Le neck profile doit être ajusté pour donner cette valeur exacte.')

In [ ]:
# BONUS: Find neck length that gives λ₁ × H* = 14
print('='*60)
print('BONUS: Reverse engineering - what T gives λ₁×H* = 14?')
print('='*60)

from scipy.optimize import brentq

def lambda_neck_for_T(T, H_star, n=500):
    """Compute λ_neck for given neck length T"""
    T0 = T / 3
    def neck_metric(x):
        return np.cosh((x - T/2) / T0)**2
    return get_lambda1_1d(n, T, neck_metric)

for name, b2, b3 in [('K7', 21, 77), ('J1', 12, 43)]:
    H_star = b2 + b3 + 1
    target_lambda = 14.0 / H_star
    
    # Find T such that λ_neck(T) = 14/H*
    def objective(T):
        return lambda_neck_for_T(T, H_star) - target_lambda
    
    # Search in range [1, 50]
    try:
        T_opt = brentq(objective, 0.5, 50)
        T_theory = np.sqrt(H_star)  # GIFT T = √H*
        ratio = T_opt / T_theory
        print(f'{name} (H*={H_star}): T_optimal = {T_opt:.4f}, T_GIFT = {T_theory:.4f}, ratio = {ratio:.4f}')
    except:
        print(f'{name}: No solution found in range [0.5, 50]')

print('\n=> Si ratio ≈ 1, le modèle TCS avec T=√H* donne naturellement λ₁×H*=14')